#### Step1: make folder to store all tickers news data


In [7]:
mkdir csv_files

A subdirectory or file csv_files1 already exists.


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import requests
import json
import datetime
import time
import os
import re
import nltk
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import yfinance as yf
import random

In [22]:
def datetime_to_date(df):
    # Drop the 'image' column
    if 'image' in df.columns:
        df = df.drop('image', axis=1)
    if 'source' in df.columns:
        df = df.drop('source', axis=1)
    if 'url' in df.columns:
        df = df.drop('url', axis=1)
    if 'id' in df.columns:
        df = df.drop('id', axis=1)
    # Convert epoch time to date for 'datetime' column
    # df = df.sort_values('datetime')
    df = df.reset_index(drop=True)
    df = df.sort_values('datetime')
    df['datetime'] = df['datetime'].apply(lambda x: pd.to_datetime(x, unit='s').date())
    df = df.reset_index(drop=True)
    
    return df

### Play with dates
start_date = datetime.strptime("2021-09-01", "%Y-%m-%d").date()

end_date = datetime.strptime("2023-05-01", "%Y-%m-%d").date()

make end date before 2020
and start before 2018 you will not see anydata , as not able to fetch


Ek jupyter file me laga k rakh lo aur fir kuch aur kaam kro preprocessing pe

In [11]:
!pip install finnhub

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement finnhub (from versions: none)
ERROR: No matching distribution found for finnhub


In [31]:
import finnhub
# Setup client
finnhub_client = finnhub.Client(api_key="ch9l37pr01qrd9soe8j0ch9l37pr01qrd9soe8jg")
finnhub_client

In [ ]:
# List of ticker symbols
ticker_symbol = ['TSLA','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX',
                 'SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']

for ticker in ticker_symbol:
    # for each ticker
    print(ticker)
    start_date = datetime.strptime("2021-09-01", "%Y-%m-%d").date()
    end_date = datetime.strptime("2023-05-01", "%Y-%m-%d").date()
    first_iteration = True
    #create file me column name to write once then append
    while(start_date <= end_date):
        jump_date = start_date + timedelta(days=10)
        a = finnhub_client.company_news(ticker, _from=start_date, to=jump_date)
        df = pd.DataFrame(a)
        start_date = jump_date
        # Wait for 1 second to avoid being blocked by Google
        time.sleep(2)
        # append to file
        if not df.empty:
            if first_iteration:
                with open(f'{ticker}1.csv',mode ='w') as file:
                # append the dataframes accordingly
                    df.to_csv(file, header=True, index=False)
                    first_iteration = False
            else:
                with open(f'{ticker}1.csv',mode ='a') as file:
                    df.to_csv(file,header=False, index=False)

    df = pd.read_csv(f'{ticker}1.csv')
    df = datetime_to_date(df)
    df = df.to_csv(f'csv_files/{ticker}1.csv')
    time.sleep(10)

## now have to attach stock rates to each

In [1]:
mkdir stock_new_data

In [32]:
# create a dictionary of date, open, and close values for a stock
def get_stock_data(ticker, start_date = "2020-05-02", end_date = "2023-05-04"):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_dict = {}
    for date, data in stock_data.iterrows():
        stock_dict[date.strftime('%Y-%m-%d')] = {"Open": data["Open"], "Close": data["Close"]}
    return stock_dict

In [23]:
def remove_duplicacy(df):
    df = df.drop_duplicates(subset=['headline'])
    df = df.drop_duplicates(subset=['summary'])
    df = df.drop(columns='category',axis=1)
    return df

In [ ]:
ticker_symbol = ['TSLA','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX',
                 'SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']
# ticker_symbol = ['ALGN']
for ticker in ticker_symbol:
    df = pd.read_csv(f'unprocessed/{ticker}.csv')
    df = datetime_to_date(df)
    df = remove_duplicacy(df)
    stock_dict = get_stock_data(ticker)
    # print(stock_dict)
    for row in df.itertuples():
        start_date = row.datetime
        if str(start_date) in stock_dict:
            df.loc[row.Index, "Open"] = stock_dict[str(start_date)]["Open"]
            df.loc[row.Index, "Close"] = stock_dict[str(start_date)]["Close"]
            if df.loc[row.Index, "Close"] > df.loc[row.Index, "Open"]:
                df.loc[row.Index, "result"] = 1
                # df["result"] = df["result"].astype(int)
            else:
                df.loc[row.Index, "result"] = 0
    df = df.dropna(how='any')
    #for latest as stock is not there
    df.to_csv(f'stock_new_data/{ticker}.csv',index=False)
    # print(stock_dict)

In [61]:
a = pd.read_csv("stock_new_data/TSLA.csv")
a

,datetime,headline,related,summary,Open,Close,result
0,2022-05-03,Who Is Elon Musk? What Is His Claim to Fame?,TSLA,Entrepreneur Elon Musk has achieved global fam...,301.059998,303.083344,1.0
1,2022-05-19,EV Start-Ups Need Cash. Lucid Is Getting More ...,TSLA,Lucid is getting financing and incentives for ...,235.666672,236.473328,1.0
2,2022-05-19,Why Tesla Fan Dan Ives Just Slashed His Price ...,TSLA,Dan Ives has reduced his TSLA stock price targ...,235.666672,236.473328,1.0
3,2022-05-19,Mortgage Interest Rates Price Predictions: Whe...,TSLA,As inflation pushes mortgage interest rates hi...,235.666672,236.473328,1.0
4,2022-05-19,Tesla’s Removal From S&P Index Sparks Debate A...,TSLA,(Bloomberg) -- A benchmark ESG stock index has...,235.666672,236.473328,1.0
...,...,...,...,...,...,...,...
3821,2023-05-03,Ford v Tesla: The EV price war continues,TSLA,"While other automakers have balked, Ford is ro...",160.009995,160.610001,1.0
3822,2023-05-03,UPDATE 2-Ford CEO calls electric vehicle marke...,TSLA,Ford CEO Jim Farley said on Wednesday that pri...,160.009995,160.610001,1.0
3823,2023-05-03,"'I Think People Can Make It On $999 Million,' ...",TSLA,It’s no secret that U.S. Sen. Bernie Sanders i...,160.009995,160.610001,1.0
3824,2023-05-03,Ford CEO calls electric vehicle market price c...,TSLA,Ford CEO Jim Farley said on Wednesday that pri...,160.009995,160.610001,1.0


# now filling the vacant dates with newst 1/30 news and merge

In [68]:
ticker_symbol = ['TSLA','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX','SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']

first_iteration = True
df=pd.DataFrame()
for ticker in ticker_symbol:
    a = pd.read_csv(f'stock_new_data/{ticker}.csv')
    if first_iteration:
        df = a
        first_iteration=False
    else:
        df = df.append(a)
df.to_csv("a.csv",index=False)



# Now i want a fill all those dates where not dates


In [37]:
def stock_date_range(df):
    # Assuming your existing DataFrame is called df and the datetime column is named 'datetime'
    start_date = pd.to_datetime(df.loc[0, 'datetime'])
    end_date = pd.to_datetime(df.loc[df.shape[0]-1, 'datetime'])

    # Create a DataFrame with all dates between start_date and end_date
    date_range = pd.date_range(start_date, end_date, freq='D').to_list()
    date_df = pd.DataFrame({'date': date_range})
    
    # Convert the date column to a list of strings
    date_list = date_df['date'].astype(str).tolist()
    # print(date_list)
    return date_list

In [38]:
def stock_unique_dates(df):
    unique_dates = df['datetime'].unique().tolist()
    # print(unique_dates)
    return unique_dates

def update_dataframe(df,dates_left):
    
    

In [26]:
def update_dataframe(df,dates_left):
    for _date in dates_left:
    # Find the row in df with the datetime closest to _date
        row = df.loc[(df['datetime'] - pd.to_datetime(_date)).abs().idxmin()]

        # Create a new DataFrame with just the _date and the other columns from the row
        new_row = pd.DataFrame({
            'datetime': [_date],
            'headline': [row['headline']],
            'related': [row['related']],
            'summary': [row['summary']],
            'Open': [row['Open']],
            'Close': [row['Close']],
            'result': [row['result']]
        })

        # Append the new row to the original DataFrame
        df = df.append(new_row, ignore_index=True)

    # Print the updated DataFrame
    print(df)
    
    


## very bad results

In [72]:
df= pd.read_csv("stock_new_data/ALGN.csv")
# print(df.head())
for _date in gap_list:
    # Find the row in df with the datetime closest to _date
    # print(df['datetime'])
    _date = pd.to_datetime(_date).strftime('%Y-%m-%d')
    # print(_date)
    row = df.loc[(pd.to_datetime(df['datetime']) - pd.to_datetime(_date)).abs().idxmin()]


    # Create a new DataFrame with just the _date and the other columns from the row
    new_row = pd.DataFrame({
        'datetime': [_date],
        'headline': [row['headline']],
        'related': [row['related']],
        'summary': [row['summary']],
        'Open': [row['Open']],
        'Close': [row['Close']],
        'result': [row['result']]
    })

    # Append the new row to the original DataFrame
    df = df.append(new_row, ignore_index=True)

# Print the updated DataFrame
print(df.head())
# update_dataframe(df,gap_list)

print(stock_unique_dates(df))
print(len(stock_unique_dates(df)))
df.to_csv("temp1.csv", index=False)

     datetime                                           headline related  \
0  2022-05-09  7 Dental Stocks to Buy as Patients Face the Dr...    ALGN   
1  2022-05-09  Cable companies to offer effectively free inte...    ALGN   
2  2022-05-09  The Smart Money Is Putting a Bullish Spin on t...    ALGN   
3  2022-05-09  Align Technology Inc. stock underperforms Mond...    ALGN   
4  2022-05-10  Align Technology Inc. stock underperforms Tues...    ALGN   

                                             summary        Open       Close  \
0  The two years away from the “chair” could cyni...  274.220001  268.959991   
1  The news means that tens of millions of househ...  274.220001  268.959991   
2  We’re seeing a bullish indicator -- another in...  274.220001  268.959991   
3  Shares of Align Technology Inc. shed 4.08% to ...  274.220001  268.959991   
4  Shares of Align Technology Inc. shed 4.03% to ...  273.500000  258.130005   

   result  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4 

## very bad results
### instead of randomly select the minimum we have to use random and select from min 10 news surrounding

In [39]:
import random

In [50]:
def fill_dataframe1(df, dates_left):
    new_df = pd.DataFrame(columns=['datetime', 'headline', 'related', 'summary', 'Open', 'Close', 'result'])
    for _date in dates_left:
        nearest_rows = df.loc[(pd.to_datetime(df['datetime']) - pd.to_datetime(_date)).abs().argsort()[:15]]
        row = nearest_rows.iloc[random.randint(0, min(len(nearest_rows), 15) - 1)]
        new_row = {'datetime': _date, 'headline': row['headline'], 'related': row['related'], 'summary': row['summary'], 'Open': row['Open'], 'Close': row['Close'], 'result': row['result']}
        new_df = new_df.append(new_row, ignore_index=True)
    return new_df


In [59]:
df= pd.read_csv("stock_new_data/ALGN.csv")
gap_list = stock_date_range(df)
# print(df.head())
new_df = pd.DataFrame(columns=['datetime', 'headline', 'related', 'summary', 'Open', 'Close', 'result'])
for _date in gap_list:
    nearest_rows = df.loc[(pd.to_datetime(df['datetime']) - pd.to_datetime(_date)).abs().argsort()[:15]]
    row = nearest_rows.iloc[random.randint(0, min(len(nearest_rows), 15) - 1)]
    new_row = {'datetime': _date, 'headline': row['headline'], 'related': row['related'], 'summary': row['summary'], 'Open': row['Open'], 'Close': row['Close'], 'result': row['result']}
    new_df = new_df.append(new_row, ignore_index=True)
new_df
 

,datetime,headline,related,summary,Open,Close,result
0,2022-05-09,Align Technology Inc. stock underperforms Mond...,ALGN,Shares of Align Technology Inc. shed 4.08% to ...,274.220001,268.959991,0.0
1,2022-05-10,Align Technology Inc. stock outperforms market...,ALGN,Shares of Align Technology Inc. rose 6.16% to ...,264.420013,275.519989,1.0
2,2022-05-11,7 Dental Stocks to Buy as Patients Face the Dr...,ALGN,The two years away from the “chair” could cyni...,274.220001,268.959991,0.0
3,2022-05-12,Align Technology Inc. stock outperforms market...,ALGN,Shares of Align Technology Inc. rose 6.16% to ...,264.420013,275.519989,1.0
4,2022-05-13,"Align Technology Inc. stock rises Tuesday, out...",ALGN,Shares of Align Technology Inc. rose 2.33% to ...,279.959991,280.589996,1.0
...,...,...,...,...,...,...,...
355,2023-04-29,These 2 Stocks Moved the Nasdaq After Hours,ALGN,Investors remained on uncertain footing on Wed...,321.950012,317.730011,0.0
356,2023-04-30,These 2 Stocks Moved the Nasdaq After Hours,ALGN,Investors remained on uncertain footing on Wed...,321.950012,317.730011,0.0
357,2023-05-01,Invisalign maker Align Tech stock falls more t...,ALGN,Shares of Align Technology Inc. dropped more t...,351.829987,354.549988,1.0
358,2023-05-02,Stocks making the biggest moves after hours: M...,ALGN,These are the stocks posting the largest moves...,351.829987,354.549988,1.0


### So every time new data comes so a very handy method

In [41]:
mkdir complete_year_wise_data

A subdirectory or file complete_year_wise_data already exists.


### here we are filling with random 1 from closest 15 nearby to the date

In [80]:
def fill_dataframe(df, dates_left):
    print(dates_left)
    pass
    new_df = pd.DataFrame(columns=['datetime', 'headline', 'related', 'summary', 'Open', 'Close', 'result'])
    for _date in dates_left:
        nearest_rows = df.loc[(pd.to_datetime(df['datetime']) - pd.to_datetime(_date)).abs().argsort()[:15]]
        row = nearest_rows.iloc[random.randint(0, min(len(nearest_rows), 15) - 1)]
        new_row = {'datetime': _date, 'headline': row['headline'], 'related': row['related'], 'summary': row['summary'], 'Open': row['Open'], 'Close': row['Close'], 'result': row['result']}
        new_df = new_df.append(new_row, ignore_index=True)
        # break
    return new_df


In [ ]:
ticker_symbol = ['TSLA','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX','SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']
#for each ticker
for ticker in ticker_symbol:
    print(ticker)
    #read ticker data of stock 
    df = pd.read_csv(f'stock_new_data/{ticker}.csv')

    # find range of date present  in ticker.csv file actaully
    date_range = stock_date_range(df)
    # print(date_range)
     # finduniquw dates data present in ticker.csv file actaully
    unique_dates = stock_unique_dates(df) 
    # print(unique_dates)

    # Convert the date strings to sets
    date_set1 = set(date_range)
    date_set2 = set(unique_dates)

    # Find the difference between the sets
    stock_empty_dates = date_set1 - date_set2

    # Convert the result back to a list
    stock_empty_dates = list(stock_empty_dates)
    # print(stock_empty_dates)
    new_df = fill_dataframe(df, stock_empty_dates)
    # concatenate vertically
    result = pd.concat([df, new_df], axis=0)
    # Append the new row to the original DataFrame
    # break
    result = result.sort_values('datetime')
    result.to_csv(f"complete_year_wise_data/{ticker}.csv",index=False)


In [77]:
df = pd.read_csv("complete_year_wise_data/ALGN.csv")

In [78]:
df = df.drop_duplicates(subset=['datetime'], keep='first')
df.to_csv("complete_year_wise_data/ALGN.csv")